# 03 - YOLO video demo for work zone detection

This notebook loads the trained YOLO model (`best.pt`), runs it on a video,
computes a simple `work_zone_score` per frame, overlays boxes and a banner,
and writes an annotated video file.


In [1]:
from pathlib import Path
import cv2
import numpy as np
from ultralytics import YOLO
from tqdm.auto import tqdm

# Paths
project_dir = Path.cwd()  # adjust if needed
run_dir = project_dir / "runs_workzone" / "yolov8s_workzone"  # or your actual run
best_ckpt = run_dir / "weights" / "best.pt"

# Input and output videos
input_video = project_dir / "data" / "demo" / "boston_workzone_short.mp4"  # change path
output_dir = project_dir / "runs_workzone" / "video_demos"
output_dir.mkdir(parents=True, exist_ok=True)
output_video = output_dir / "boston_workzone_annotated.mp4"

# Detection settings
conf_thres = 0.4
iou_thres = 0.5

print("Best checkpoint:", best_ckpt)
print("Input video:", input_video)
print("Output video:", output_video)

Best checkpoint: c:\Users\wesle\Music\workingzone\runs_workzone\yolov8s_workzone\weights\best.pt
Input video: c:\Users\wesle\Music\workingzone\data\demo\boston_workzone_short.mp4
Output video: c:\Users\wesle\Music\workingzone\runs_workzone\video_demos\boston_workzone_annotated.mp4


c:\Users\wesle\anaconda3\envs\slt311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = YOLO(str(best_ckpt))
names = model.model.names  # class index -> name
print("Classes:", names)

Classes: {0: 'Cone', 1: 'Drum', 2: 'Barricade', 3: 'Barrier', 4: 'Vertical Panel', 5: 'Work Vehicle', 6: 'Worker', 7: 'Arrow Board', 8: 'Temporary Traffic Control Message Board', 9: 'Temporary Traffic Control Sign'}


In [3]:
# Define which classes indicate work zone and their weights
workzone_weights = {
    "Cone": 1.0,
    "Drum": 1.2,
    "Barricade": 1.5,
    "Barrier": 1.0,
    "Vertical Panel": 1.0,
    "Work Vehicle": 2.0,
    "Worker": 2.5,
    "Arrow Board": 1.5,
    "Temporary Traffic Control Message Board": 1.5,
    "Temporary Traffic Control Sign": 1.2,
}

def compute_work_zone_score(result):
    """
    result is a single ultralytics Result object for one frame.
    Returns a scalar score in [0, 1] (roughly).
    """
    if result.boxes is None or len(result.boxes) == 0:
        return 0.0
    
    cls_ids = result.boxes.cls.cpu().numpy().astype(int)
    confs = result.boxes.conf.cpu().numpy()
    
    score = 0.0
    max_possible = 0.0
    
    for cls_id, conf in zip(cls_ids, confs):
        cls_name = names.get(cls_id, str(cls_id))
        weight = workzone_weights.get(cls_name, 0.0)
        score += weight * float(conf)
        max_possible += weight
    
    if max_possible == 0.0:
        return 0.0
    # Normalize to [0, 1] by dividing by max_possible and clipping
    return float(np.clip(score / max_possible, 0.0, 1.0))

In [5]:
cap = cv2.VideoCapture(str(input_video))
assert cap.isOpened(), f"Cannot open video {input_video}"

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(output_video), fourcc, fps, (width, height))

print(f"Video info: {width}x{height} at {fps:.1f} fps, {total_frames} frames")

frame_scores = []

for _ in tqdm(range(total_frames), desc="Processing video"):
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run YOLO on current frame
    results = model(frame, conf=conf_thres, iou=iou_thres, verbose=False)
    result = results[0]
    
    # Compute scene score
    score = compute_work_zone_score(result)
    frame_scores.append(score)
    
    # Get image with bounding boxes drawn by Ultralytics
    annotated = result.plot()  # returns BGR numpy array
    
    # Compose banner text
    if score > 0.6:
        label_text = f"WORK ZONE - score {score:.2f}"
        banner_color = (0, 0, 255)  # red
    elif score > 0.3:
        label_text = f"POSSIBLE WORK ZONE - score {score:.2f}"
        banner_color = (0, 165, 255)  # orange
    else:
        label_text = f"NO WORK ZONE - score {score:.2f}"
        banner_color = (0, 255, 0)  # green
    
    # Draw banner rectangle at top
    banner_height = int(0.08 * height)
    cv2.rectangle(
        annotated,
        (0, 0),
        (width, banner_height),
        banner_color,
        thickness=-1,
    )
    
    # Put text
    cv2.putText(
        annotated,
        label_text,
        (20, int(banner_height * 0.7)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,
        (255, 255, 255),
        2,
        cv2.LINE_AA,
    )
    
    # Write to output
    writer.write(annotated)

cap.release()
writer.release()

print("Saved annotated video to:", output_video)


Video info: 722x406 at 30.0 fps, 451 frames


Processing video:   0%|          | 0/451 [00:00<?, ?it/s]

Processing video: 100%|██████████| 451/451 [00:14<00:00, 32.13it/s]

Saved annotated video to: c:\Users\wesle\Music\workingzone\runs_workzone\video_demos\boston_workzone_annotated.mp4
